## Chapter 16: Advanced Function Features of Julia

This chapter covers a bit more on functions in julia. These feature allow the ability to write code that easier to use, read and debug. We will perform error checking for arguments to ensure that only valid arguments are considered. Additionally, to make functions more robust, we’ll use option arguments and keyword arguments.

#### 16.1: Testing Arguments

Recall the recursive factorial function:

In [ ]:
fact(n::Integer) = n==0 ? 1 : n*fact(n-1)

If we put in a negative number, then

In [ ]:
fact(-5)

This occurs because fact(-5) would return -5 times fact(-6) which would return -5 times -6 times fact(-7), etc.  This will never stop. 

To prevent this, we will check if the argument n is greater than or equal to 0. The first line of the function, evaluates `n>=0`. If this is true, it skips the rest of the line.  If not, it throws an error. 

In [ ]:
function fact(n::Integer)
  n>=0 || throw(ArgumentError("The argument must be a nonnegative integer."))
  n==0 ? 1 : n*fact(n-1)
end

In [ ]:
fact(-5)

#### 16.2: Optional arguments

Recall Newton's method that we saw earlier:

In [ ]:
using ForwardDiff
function newton(f::Function, x0::Number)
  local dx = f(x0)/ForwardDiff.derivative(f,x0) 
  local steps = 0
  while abs(dx)>1e-6 && steps<10
    x0 -= x0
    dx = f(x0)/ForwardDiff.derivative(f,x0) 
    steps += 1
  end 
end

Note: we haven't used the `Root` datatype developed in Chapter 12 for simplicity.  An exercise would be to combine this.

What if we want the root tolerance (1e-6) to be better or to run more steps than 10.  These are hard coded parameters that would have to be updated in the newton function.  A much better way to do this is to add these to the argument list, but with default values.  Julia calls these optional arguments.  Consider this rewrite:

In [ ]:
function newton(f::Function, x0::Number, tol=1e-6, max_steps=10)
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 = x0 - dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

First, notice that there are 3 methods now. What are the signatures of newton for this?

Now, notice that we can change the tolerance with:

In [ ]:
newton(x->x^2-5,1,1e-12)

But a problem is that to change the maximum number of steps, we need to change both:

In [ ]:
newton(x->x^2-5,1,1e-12,20)

We don't need to set the argument types for optional arguments.  Julia will figure out the type by the number that you put in.  The example above, `tol` will be a Float and `max_steps` will be an Integer. 

We should also check these new arguments have proper types (each should be positive) with:

In [ ]:
function newton(f::Function, x0::Number,tol=1e-6,max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol must be positive"))
  max_steps > 0 || throw(ArgumentError("The parameter max_steps must be positive"))
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 = x0 - dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  x0
end

In [ ]:
newton(x->x^2-5,1,-0.01)

#### 16.3: Handling Special Cases

In [ ]:
newton(x->x^2+2,1)

In [ ]:
using ForwardDiff
function newton(f::Function, x0::Number,tol=1e-6,max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol must be positive"))
  max_steps > 0 || throw(ArgumentError("The parameter max_steps must be positive"))
  local dx = f(x0)/ForwardDiff.derivative(f,x0)
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  local error = "The maximum number of steps: $max_steps was reached without convergence"
  steps < max_steps || throw(ErrorException(error))
  x0
end

In [ ]:
newton(x->x^2+2,1)

#### 16.4: Keyword Arguments

Instead of optional arguments (especially if there are more than one), it often better to use keyword arugments in which you can change any of the arguments in any order. To do this, we need to use the keyword when calling the function.  To change from optional arguments to keyword arguments, separate the regular arguments from the keyword ones with a semicolon (;)

In [ ]:
function newton(f::Function, x0::Number; tol=1e-6, max_steps=10)
  tol > 0 || throw(ArgumentError("The parameter tol much be positive")) 
  max_steps > 0 || throw(ArgumentError("The parameter max_steps much be positive"))
  local x1 = x0
  local dx = f(x0)/ForwardDiff.derivative(f,x0) 
  local steps = 0
  while abs(dx)>tol && steps<max_steps
    x0 -= dx
    dx = f(x0)/ForwardDiff.derivative(f,x0)
    steps += 1
  end
  local error = "The maximum number of steps: $max_steps was reached without convergence"
  steps < max_steps || throw(ErrorException(error))
  x0
end

In [ ]:
newton(x->x^2-5,1,max_steps=5)

#### 16.5: Parametric Methods

In Chapter 4, we saw multiple dispatch for julia.  Let's look at another example of this.  This will find the maximum value with different arguments put in. 

If we have two values we can do:

In [ ]:
findMax(x::Real,y::Real) = x > y ? x : y

In [ ]:
findMax(3,4)

As as we saw earlier, if we want any number of arguments, we could do:

In [ ]:
function findMax(nums::Real...)
  local max = -Inf
  for num in nums
    if num > max
      max = num
    end
  end
  max
end

In [ ]:
findMax(1,6,2,3,-9,11//2,5.6)

It makes sense to also write a `findMax` function that takes an array of values:

In [ ]:
function findMax(arr::Vector{Real})
  local max = -Inf
  for num in arr
    if num > max
      max = num
    end
  end
  max
end

Let's say we want to find the max of `[-3,2,4,7,5]`.  It seems like the following should work:

In [ ]:
findMax([-3,2,4,7,5])

However, notice that the error says that it can't find a `findMax` function with argument `Array{Int64,1}`.  

We could create functions to handle arrays of integers, floats, and other types that we need, but julia has what is called parametric methods that handles all of these together:

In [ ]:
function findMax(arr::Vector{T}) where {T <: Real}
  local max = -Inf
  for num in arr
    if num > max
      max = num
    end
  end
  max
end

which can now handle any array with any subtype of `Real`.

In [ ]:
findMax([-3,2,4,7,5])

In [ ]:
typeof([-3,2,4,7,5])

In [ ]:
findMax([pi,1.0,-6.3, 6/8])

In [ ]:
findMax([big(3),big(9)])